In [ ]:
%matplotlib widget
import pandas as pd
import polars as pl
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.colors as colors

from scipy.stats.mstats import kruskalwallis, mannwhitneyu
import scikit_posthocs

In [ ]:
white_headed_breeds = ['cross','MON','NMD','SIM','FV','HER','GWH','YAR'] + ['cSIM','SxH','FLK','VWD','HWD','sHER','KWH']
info = pl.read_csv('sample_information.csv',separator=' ')
nodes = pl.read_csv('subgraph.nodes.csv',separator=' ')
KIT_nodes = [int(i) for i in open('KIT_nodes')]
coverage = (pl.read_csv('node_coverage.csv.gz',separator=' ')
                  .join(info,on='sample')
                  .join(nodes,on='node')
                  .with_columns([pl.col('breed').is_in(white_headed_breeds).alias('White-headed'),pl.col('node').is_in(KIT_nodes).alias('KIT'),(pl.col('coverage')*pl.col('length')).alias('total coverage')])
           )
x_order = [i for i in coverage['breed'].unique() if i not in white_headed_breeds] + white_headed_breeds

In [ ]:
averages_r = (coverage.filter((pl.col('length')>=0)&(pl.col('coverage')>=0))
                      .group_by('sample','breed','KIT','coverage_right','duplication_rate')
                      .agg(pl.col('total coverage').sum())
                      .with_columns([(pl.col('total coverage')/(pl.col('coverage_right')*(1-pl.col('duplication_rate')))).alias('adjusted coverage')])
                      .with_columns([(pl.col('adjusted coverage')/((14325*pl.col('KIT') + 2e6*(~pl.col('KIT'))))).alias('Length normalised coverage')])
             )
averages = averages_r.to_pandas()

g = sns.catplot(data=averages,x='breed',y='Length normalised coverage',hue='breed',col='KIT',order=x_order)
g.map(sns.boxplot,'breed','Length normalised coverage',order=x_order,**{'boxprops':{'facecolor':'none'}})
g.set_xticklabels(rotation=45,ha='center')
for ax in g.axes_dict.values():
    ax.axline((0,1), slope=0, c=".2", ls="--", zorder=0)

In [ ]:
p_matrix = scikit_posthocs.posthoc_mannwhitney(averages_r.filter(pl.col('KIT')).to_pandas(),'Length normalised coverage','breed',p_adjust='bonferroni')
plt.figure()
scikit_posthocs.sign_plot(p_matrix[x_order].reindex(index = x_order).to_numpy())

f, ax = plt.subplots()

p = ax.pcolormesh(p_matrix[x_order].reindex(index = x_order).to_numpy(),norm=colors.LogNorm(vmin=1e-10, vmax=1))
f.colorbar(p)
ax.set_yticks(np.arange(len(p_matrix.index))+0.5)
ax.set_yticklabels(p_matrix.index)

ax.set_xticks(np.arange(len(p_matrix.columns))+0.5)

ax.set_xticklabels(p_matrix.columns,rotation=45,ha='center')

plt.show()

In [ ]:
def map_rolling(data,x,y,color,**kws):
    ax = plt.gca()
    ax.plot(data[x],data[y].rolling(10).mean(),color=color)

def label(x, color, label):
    ax = plt.gca() #get current axis
    ax.text(-.1, .4, label, color='black', fontsize=13,
            ha="left", va="center", transform=ax.transAxes)

In [ ]:
## rolling node plot

means = (coverage.filter((pl.col('KIT'))&(pl.col('length')>=0)&(pl.col('coverage')>=0))
                      .with_columns([(pl.col('coverage')/((pl.col('coverage_right')*(1-pl.col('duplication_rate'))))).alias('adjusted coverage')])
                      .group_by('breed','node')
                      .agg(pl.col('adjusted coverage').median(),pl.col('length').median())
                      .sort(by='node')
             ).to_pandas()

g = sns.FacetGrid(means, hue='breed',height=.75,aspect=10,row="breed",row_order=x_order)
g.map_dataframe(map_rolling,x='node',y='adjusted coverage')
#g.refline(y=0, linewidth=0.5, linestyle="-", color=None, clip_on=False)
g.figure.subplots_adjust(hspace=-5.75)
g.set_titles("")
g.map(label, "breed")
g.set(yticks=[], ylabel="")
g.despine(bottom=False, left=True)

In [ ]:
### Sample outlier detection

## Weird FV sample
print(averages[(averages['coverage']>0.90)&(averages['KIT']==False)&(averages['breed']=='FV')])


## removing bad CHI/CHA samples with large insert sizes and stddev
bad_inserts = ['UMCUSAU000000194426','UMCUSAU000000194423','UMCUSAU000000194424','UMCUSAU000000194748','UMCUSAU000000194761','UMCUSAU000000194425','UMCUSAU000000194370']
print(averages[(averages['coverage']<0.25)&(averages['KIT']==False)&(averages['sample'].isin(bad_inserts))])


## Potentially a mislabelling for SAMEA7690196
print(averages[(averages['coverage']>0.60)&(averages['KIT']==True)&(averages['breed']=='CHI')])

## same as first weird FV sample
print(averages[(averages['coverage']>1.60)&(averages['KIT']==True)&(averages['breed']=='FV')])

print(averages[(averages['coverage']>2.00)&(averages['KIT']==True)&(averages['breed']=='SIM')])

print(averages[(averages['coverage']<.20)&(averages['KIT']==True)&(averages['breed']=='NMD')])

print(averages[(averages['coverage']>4.5)&(averages['KIT']==True)&(averages['breed']=='HER')])

In [ ]:
## DEAD CODE
averages = coverage.filter((pl.col('length')>=0)&(pl.col('coverage')>=0)).group_by('sample','breed','KIT','coverage_right','duplication_rate').agg(pl.col('coverage').mean()).to_pandas()
averages['coverage']/=(averages['coverage_right']*(1-averages['duplication_rate']))
g = sns.catplot(data=averages,x='breed',y='coverage',hue='breed',col='KIT',order=x_order)
g.map(sns.boxplot,'breed','coverage',order=x_order,**{'boxprops':{'facecolor':'none'}})
g.set_xticklabels(rotation=45,ha='center')


averages = coverage.group_by('node','breed','KIT','coverage_right','duplication_rate').agg(pl.col('coverage').mean()).to_pandas()
averages['coverage']/=(averages['coverage_right']*(1-averages['duplication_rate']))
sns.lmplot(data=means[means['cov']>0.05],x='node',y='cov',hue='breed',col='breed',col_order=x_order,col_wrap=3,lowess=True,scatter_kws={'s':20})
sns.catplot(data=x,kind='point',x='node',y='adjusted coverage',hue='breed',col='breed',col_order=x_order,col_wrap=3)

import statsmodels.api as sm
lowess = sm.nonparametric.lowess
from scipy.interpolate import splev, splrep, UnivariateSpline

def map_lowess(data,x,y,color):
    spl = UnivariateSpline(list(data['node']), list(data['cov']),w=np.sqrt(data['length']),s=0)#,s=5e3)
    y2 = spl(data['node'])
    ax = plt.gca()
    ax.plot(data[x],y2)
    
    (averages_r.filter(pl.col('KIT')).groupby("breed")
        .agg(krusk=pl.reduce(kruskalwallis,('adjusted coverage')))
)